In [2]:
import sys, os
from pathlib import Path
import llm, sqlite_utils

In [3]:
print("Environment check goes brrr...")
assert(sys.executable.split('/')[-3] == 'conda-env')
assert(os.environ.get('ENVIRONMENT_LOADED') == 'roombot')

Environment check goes brrr...


In [4]:
print("Database create (or load) goes brrr...")

print("   ...brrr... creating path if not exist")
db_dir = "./db/"
Path(db_dir).mkdir(parents=True, exist_ok=True)

print("   ...brrr... loading db")
db_file = db_dir + "embeddings.db"
db = sqlite_utils.Database(db_file)

Database create (or load) goes brrr...
   ...brrr... creating path if not exist
   ...brrr... loading db


In [5]:
print("Embeddings collection create (or load) goes brrr...")

print("   ...brrr... getting embedding model")
model_id = "3-small"
model = llm.get_embedding_model(model_id)

print("   ...brrr... loading (or creating) embedding collection from db")
collection_name = "code-collection"
collection = llm.Collection(collection_name, db, model=model)

Embeddings collection create (or load) goes brrr...
   ...brrr... getting embedding model
   ...brrr... loading (or creating) embedding collection from db


In [7]:
print("Embedding goes brrr...")

print("    ...brrr... Finding code to embed")
code_dir = os.environ['ROOMBOT_CODE_DIR'].rstrip('/') + '/'
print("        ...brr... using directory:", code_dir)
print("            ...brr... TODO: ignoring directory and using hardcoded values")
to_embed = [
    # id, code, metadata
    ("1", "// good code", {}),
    ("2", "// bad code", {})
]

print("    ...brrr... Embedding into db collection (overwrite all existing)")
print("        ...brrr... OpenAI money decreasing")
collection.embed_multi_with_metadata(to_embed, store=True)

Embedding goes brrr...
    ...brrr... Finding code to embed
        ...brr... using directory: ../yawa/src/
            ...brr... TODO: ignoring directory and using hardcoded values
    ...brrr... Embedding into db collection (overwrite all existing)
        ...brrr... OpenAI money decreasing


In [8]:
print("Searching embeddings goes brrr...")
print("    ...brrr... OpenAI money decreasing")
query = "needs refactoring"
for entry in collection.similar(query, number=1):
    print("\n\nENTRY ID and SCORE:")
    print(entry.id, entry.score)
    print("\n\nCODE:")
    print(entry.content)
    print("\n\nMETADATA:")
    print(entry.metadata)

Searching embeddings goes brrr...
    ...brrr... OpenAI money decreasing


ENTRY ID and SCORE:
2 0.3185897734199054


CODE:
// bad code


METADATA:
None
